<a href="https://colab.research.google.com/github/vishalveerareddy/Word2VecGOT/blob/master/Word2Vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
! git clone "https://github.com/llSourcell/word_vectors_game_of_thrones-LIVE.git"

Cloning into 'word_vectors_game_of_thrones-LIVE'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23
Unpacking objects: 100% (23/23), done.


In [0]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

In [0]:

import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [0]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [0]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)


In [0]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
book_filenames = sorted(glob.glob("/content/word_vectors_game_of_thrones-LIVE/data/*.txt"))

In [0]:
print("Found books:")
book_filenames

Found books:


['/content/word_vectors_game_of_thrones-LIVE/data/got1.txt',
 '/content/word_vectors_game_of_thrones-LIVE/data/got2.txt',
 '/content/word_vectors_game_of_thrones-LIVE/data/got3.txt',
 '/content/word_vectors_game_of_thrones-LIVE/data/got4.txt',
 '/content/word_vectors_game_of_thrones-LIVE/data/got5.txt']

In [0]:
corpus_raw = u""
for book_filename in book_filenames:
    print("Reading '{0}'...".format(book_filename))
    with codecs.open(book_filename, "r", "utf-8") as book_file:
        corpus_raw += book_file.read()
    print("Corpus is now {0} characters long".format(len(corpus_raw)))
    print()

Reading '/content/word_vectors_game_of_thrones-LIVE/data/got1.txt'...
Corpus is now 1770659 characters long

Reading '/content/word_vectors_game_of_thrones-LIVE/data/got2.txt'...
Corpus is now 4071041 characters long

Reading '/content/word_vectors_game_of_thrones-LIVE/data/got3.txt'...
Corpus is now 6391405 characters long

Reading '/content/word_vectors_game_of_thrones-LIVE/data/got4.txt'...
Corpus is now 8107945 characters long

Reading '/content/word_vectors_game_of_thrones-LIVE/data/got5.txt'...
Corpus is now 9719485 characters long



In [0]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [0]:
raw_sentences = tokenizer.tokenize(corpus_raw)


In [0]:
def sentence_to_wordlist(raw):
    clean = re.sub("[^a-zA-Z]"," ", raw)
    words = clean.split()
    return words


In [0]:
sentences = []
for raw_sentence in raw_sentences:
    if len(raw_sentence) > 0:
        sentences.append(sentence_to_wordlist(raw_sentence))

In [0]:
print(raw_sentences[5])
print(sentence_to_wordlist(raw_sentences[5]))

Heraldic crest by Virginia Norey.
['Heraldic', 'crest', 'by', 'Virginia', 'Norey']


In [0]:
token_count = sum([len(sentence) for sentence in sentences])
print("The book corpus contains {0:,} tokens".format(token_count))

The book corpus contains 1,818,103 tokens


In [0]:
#ONCE we have vectors
#step 3 - build model
#3 main tasks that vectors help with
#DISTANCE, SIMILARITY, RANKING

# Dimensionality of the resulting word vectors.
#more dimensions, more computationally expensive to train
#but also more accurate
#more dimensions = more generalized
num_features = 300
# Minimum word count threshold.
min_word_count = 3

# Number of threads to run in parallel.
#more workers, faster we train
num_workers = multiprocessing.cpu_count()

# Context window length.
context_size = 7

# Downsample setting for frequent words.
#0 - 1e-5 is good for this
downsampling = 1e-3

# Seed for the RNG, to make the results reproducible.
#random number generator
#deterministic, good for debugging
seed = 1

In [0]:
thrones2vec = w2v.Word2Vec(
    sg=1,
    seed=seed,
    workers=num_workers,
    size=num_features,
    min_count=min_word_count,
    window=context_size,
    sample=downsampling
)

In [0]:
thrones2vec.build_vocab(sentences)


2019-07-21 13:22:44,067 : INFO : collecting all words and their counts
2019-07-21 13:22:44,069 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2019-07-21 13:22:44,117 : INFO : PROGRESS: at sentence #10000, processed 140984 words, keeping 10280 word types
2019-07-21 13:22:44,155 : INFO : PROGRESS: at sentence #20000, processed 279730 words, keeping 13558 word types
2019-07-21 13:22:44,192 : INFO : PROGRESS: at sentence #30000, processed 420336 words, keeping 16598 word types
2019-07-21 13:22:44,229 : INFO : PROGRESS: at sentence #40000, processed 556581 words, keeping 18324 word types
2019-07-21 13:22:44,265 : INFO : PROGRESS: at sentence #50000, processed 686247 words, keeping 19714 word types
2019-07-21 13:22:44,306 : INFO : PROGRESS: at sentence #60000, processed 828497 words, keeping 21672 word types
2019-07-21 13:22:44,350 : INFO : PROGRESS: at sentence #70000, processed 973830 words, keeping 23093 word types
2019-07-21 13:22:44,392 : INFO : PROGRESS: at 

In [0]:

len(sentences)


128868

In [0]:
thrones2vec.train(sentences,total_examples=128868, epochs=20)

2019-07-21 13:28:03,934 : INFO : training model with 2 workers on 17277 vocabulary and 300 features, using sg=1 hs=0 sample=0.001 negative=5 window=7
2019-07-21 13:28:05,089 : INFO : EPOCH 1 - PROGRESS: at 8.20% examples, 101810 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:06,129 : INFO : EPOCH 1 - PROGRESS: at 16.05% examples, 102764 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:07,169 : INFO : EPOCH 1 - PROGRESS: at 23.78% examples, 103013 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:08,199 : INFO : EPOCH 1 - PROGRESS: at 31.78% examples, 103604 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:09,212 : INFO : EPOCH 1 - PROGRESS: at 40.20% examples, 104206 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:10,251 : INFO : EPOCH 1 - PROGRESS: at 47.71% examples, 104138 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:11,293 : INFO : EPOCH 1 - PROGRESS: at 55.13% examples, 104107 words/s, in_qsize 3, out_qsize 0
2019-07-21 13:28:12,327 : INFO : EPOCH 1 - PROGRESS: at 62.7

(28088626, 36362060)

In [0]:
if not os.path.exists("trained"):
    os.makedirs("trained")

In [0]:
thrones2vec.save(os.path.join("trained", "thrones2vec.w2v"))


2019-07-21 13:33:03,564 : INFO : saving Word2Vec object under trained/thrones2vec.w2v, separately None
2019-07-21 13:33:03,566 : INFO : not storing attribute vectors_norm
2019-07-21 13:33:03,568 : INFO : not storing attribute cum_table
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-21 13:33:04,203 : INFO : saved trained/thrones2vec.w2v


In [0]:
thrones2vec = w2v.Word2Vec.load(os.path.join("trained", "thrones2vec.w2v"))


2019-07-21 13:33:29,336 : INFO : loading Word2Vec object from trained/thrones2vec.w2v
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
2019-07-21 13:33:29,708 : INFO : loading wv recursively from trained/thrones2vec.w2v.wv.* with mmap=None
2019-07-21 13:33:29,710 : INFO : setting ignored attribute vectors_norm to None
2019-07-21 13:33:29,711 : INFO : loading vocabulary recursively from trained/thrones2vec.w2v.vocabulary.* with mmap=None
2019-07-21 13:33:29,712 : INFO : loading trainables recursively from trained/thrones2vec.w2v.trainables.* with mmap=None
2019-07-21 13:33:29,713 : INFO : setting ignored attribute cum_table to None
2019-07-21 13:33:29,714 : INFO : loaded trained/thrones2

In [0]:

#my video - how to visualize a dataset easily
tsne = sklearn.manifold.TSNE(n_components=2, random_state=0)

In [0]:
all_word_vectors_matrix = thrones2vec.syn0_lockf


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `syn0_lockf` (Attribute will be removed in 4.0.0, use self.trainables.vectors_lockf instead).
  """Entry point for launching an IPython kernel.


In [0]:
all_word_vectors_matrix.reshape(-1,1)

all_word_vectors_matrix_2d = tsne.fit_transform(all_word_vectors_matrix.reshape(-1,1))


In [0]:
thrones2vec.most_similar("Stark")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.
2019-07-21 13:45:40,102 : INFO : precomputing L2-norms of word weight vectors
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('Eddard', 0.6249960064888),
 ('executed', 0.48581451177597046),
 ('Snowbeard', 0.4553941488265991),
 ('divulge', 0.4551628530025482),
 ('fishwife', 0.4469717741012573),
 ('Winterfell', 0.4451400935649872),
 ('SHIREI', 0.44483068585395813),
 ('Edrick', 0.4339056611061096),
 ('Underfoot', 0.43169105052948),
 ('Knelt', 0.4296530783176422)]